In [1]:
cd C:/Users/Lenovo/PyDev/Retail_Chatbot/buynet/data

C:\Users\Lenovo\PyDev\Retail_Chatbot\buynet\data


In [2]:
import pandas as pd
import re
import pickle
import html2text

h = html2text.HTML2Text()
df = pd.read_json('metadata/Meta_v3.json')

In [4]:
print("Filled up columns")
for i in df.columns:
    print(i, len(df)-list(df[i]).count([]))

content_NA = 0
for i in range(len(df)):
    if df.loc[i,'description']==[] and df.loc[i, 'feature']==[]:
        content_NA += 1
print("Products with neither desciption nor features", end=": ")
print(content_NA)

no_specs = 0

for i in range(len(df)):
    if df.loc[i,'specifications']=={}:
        no_specs += 1
print("Products with no specifications", end=": ")
print(no_specs)

Filled up columns
also_buy 3199
also_view 5204
asin 11315
brand 11315
category 11315
date 11315
description 7666
feature 8477
image 9944
main_cat 11315
price 11315
rank 11143
title 11315
specifications 11315
Products with neither desciption nor features: 2460
Products with no specifications: 4413


In [7]:
def gen_content(df, i):
    content = h.handle(df.loc[i, 'title'])
    for j in range(len(df.loc[i, 'description'])):
        content = content + ' ' + h.handle(df.loc[i, 'description'][j])
    for j in range(len(df.loc[i, 'feature'])):
        content = content + ' ' + h.handle(df.loc[i, 'feature'][j])
    return content

def gen_entities(df, i):
    ents = []
    for k,v in enumerate(df.loc[i, 'specifications']):
        if v not in ents:
            ents.append(v)
    return ents

def gen_offsets(df,i):
    ents = gen_entities(df,i)
    values = []
    content = gen_content(df, i)
    final = []
    res = []
    
    for k,v in enumerate(df.loc[i, 'specifications']):
        spec_dict = df.loc[i, 'specifications']
        if len(spec_dict[v])>1:
            values.append(spec_dict[v])
    
    for j in values:
        try: 
            res.append((re.search(j.lower(), content.lower()).start(), re.search(j.lower(), content.lower()).end(), list(spec_dict.keys())[list(spec_dict.values()).index(j)]))
        except: 
            pass
    
    screen_size_match = [ r"\d\d?\.\d\"", r"\d\d?\""]
    ram_match = [r"\d\.\d\s?gb\s?ram", r"\d\s?gb\s?ram", r"\d\s?gb\s?ddr3", r"\d\s?gb,\s?ddr3", r"\d\s?gb,\s?lpddr3", r"\d\d?\s?gb\s?ddr4", r"ram:\s\d(k|g)b"]
    hdd_match = [r"\d*\s?(g|t)b\s?hdd", r"\d\s?tb "]
    ssd_match = [r"\d*\s?(g|t)b\s?ssd"]
    ghz_match = [r"\d\.\d*\s?ghz", r"\d\s?ghz", r"\d*\s?mhz"]
    screen_res_match = [r"\d\d\d\d?\s?x\s?\d\d\d\d?","\d\d\d\d?\s?\*\s?\d\d\d\d?"]
    wt_match = ["\d*\.?\d*\s?lbs", "\d*\.?\d*\s?pounds", "\d*\.?\d*\s?ounces"]
    power = ["\d*\.?\d*\s?watt"]
    transfer_speed = ["(\s|\/)\d\d?\s?(g|m)b(\/|p)s","(\s|\/)\d\d?\.\d\d?\s?(g|m)b(\/|p)s", "(\s|\/)\d\d\d\d?\s?(g|m)b(\/|p)s"]
    battery = ["\d*\s?mah"]
    waterproof = ["ipx\d"]
    sensitivity = ["\s\d*\s?db"]
    camera_resolution = ["\s\d\d?(\.\d)?\s?MP"]
    
    reg_list = [screen_size_match, ram_match, hdd_match, ssd_match, ghz_match, screen_res_match, wt_match, power, transfer_speed, battery, waterproof, sensitivity, camera_resolution]
    reg_names = ["n_size", "ram", "hdd", "ssd", "speed", "screen_resolution", "weight", "power", "transfer_speed", "battery", "waterproof_rating", "sensitivity", "camera"]
    
    for i in range(len(reg_list)):
        for j in reg_list[i]:
            if (re.search(j.lower(), content.lower()) is None):
                continue
            else:
                res.append((re.search(j.lower(), content.lower()).start(), re.search(j.lower(), content.lower()).end(), reg_names[i]))
     
    def get_ent_keys(lst):
        ents = []
        for i in lst:
            ents.append(i[2])
        return ents
    
    flash_memory = "\d\d*(\.\d)?\s?(g|t)b"
    ents = get_ent_keys(res)
    if (('RAM' not in ents) and ('hdd' not in ents) and ('ssd' not in ents) and ('Flash Memory Size' not in ents)):

        if (re.search(flash_memory, content.lower()) is None):
            pass
        else:
            res.append((re.search(flash_memory.lower(), content.lower()).start(), re.search(flash_memory.lower(), content.lower()).end(), 'memory'))
    
    return res

In [23]:
battery_match = "\d\d*\s?ma"
for i in range(len(df)):
    match = re.search(battery_match, gen_content(df, i).lower())
    if not (match is None):
        df.loc[i, 'specifications']['battery'] = match.group(0)
        continue
        
db_match = "\d\d*\s?db"
for i in range(len(df)):
    match = re.search(db_match, gen_content(df, i).lower())
    if not (match is None):
        df.loc[i, 'specifications']['sensitivity'] = match.group(0)
        continue

In [12]:
flash_memory = "\d\d*(\.\d)?\s?(g|t)b"

for i in range(len(df)):
    match = re.search(flash_memory, gen_content(df, i).lower())
    if not (match is None):
        ents = gen_entities(df, i)
        if (('RAM' not in ents) and ('hdd' not in ents) and ('ssd' not in ents) and ('Flash Memory Size' not in ents)):
            df.loc[i, 'specifications']['memory'] = match.group(0)
            continue

In [27]:
# train_data = []
# for i in range(len(df)):
#     train_data.append((gen_content(df, i), {"entities": gen_offsets(df, i)}))

In [549]:
print('1566')
TD = []
for i in range(len(train_data)):
    if (len(train_data[i][1]['entities'])>=1):
        TD.append(train_data[i])
print(len(TD))

1566
5850


In [517]:
with open("ner_train.txt", "rb") as fp:   # Unpickling
    TRAIN_DATA = pickle.load(fp)

In [550]:
jar = TD
for i in range(len(jar)):
    clx = []
    for j in range(len(jar[i][1]['entities'])):
        dup = jar[i][1]['entities'][j]
        switch = 0
        for k in range(len(clx)):
            if clx[k][2]==jar[i][1]['entities'][j][2]:
                switch = 1
        if switch == 1:
            continue
        else:
            clx.append(jar[i][1]['entities'][j])
    jar[i][1]['entities'] = clx

In [ ]:
with open("metadata/ner_train.txt", "wb") as fp:   #Pickling
    pickle.dump(TD, fp)
df.to_json('metadata/Meta_v3.json')

### Creating an aggregate dictionary

In [551]:
def gen_BD(df, i):
    MD1 = df.loc[i, 'spec_dict']
    spec_dict2 = df.loc[i, 'spec_dict2']
    content = gen_content(df, i)
    
    for i,k in enumerate(spec_dict2):
        MD1[k] = spec_dict2[k]
    ents = []
    
    for i in TD:
        if i[0] == content:
            ents = i[1]['entities']
            break
            
    for i in ents:
        MD1[i[2]]=content[i[0]:i[1]]
        
    return MD1

In [552]:
lst_dict = []
df = df.reset_index()
df = df.drop(columns="index")
for i in range(len(df)):
    lst_dict.append(gen_BD(df,i))
    
df['specifications'] = lst_dict

In [81]:
no_specs = 0
for i in range(len(df)):
    if df.loc[i,'specifications']=={}:
        no_specs += 1
print("Products with no specifications", end=": ")
print(no_specs)

Products with no specifications: 4413


In [556]:
excl = []
for i in range(len(train_data)):
    if (len(train_data[i][1]['entities'])==0):
        excl.append(train_data[i])
len(excl)

5465

In [561]:
df = df.drop(columns=['spec_dict','spec_dict2'])

In [31]:
UKS = []
rem = 0
print(len(UKS))

for i in range(len(df)):
    for v, k in enumerate(df.loc[i, 'specifications']):
        if k not in UKS:
            UKS.append(k)
            
print((UKS))

0
['waterproof_rating', 'Item Dimensions', 'sensitivity', 'Size', 'Material Type', 'Manufacturer Part Number', 'battery', 'screen_resolution', 'weight', 'transfer_speed', 'memory', 'Processor', 'RAM', 'Memory Speed', 'Hard Drive', 'Graphics Coprocessor', 'Chipset Brand', 'Card Description', 'Number of USB 2.0 Ports', 'Number of USB 3.0 Ports', 'Series', 'Hardware Platform', 'Operating System', 'Processor Brand', 'Processor Count', 'Flash Memory Size', 'Hard Drive Interface', 'Hard Drive Rotational Speed', 'Optical Drive Type', 'speed', 'Graphics Card Ram Size', 'Wireless Type', 'Computer Memory Type', 'Display Size', 'Item Package Quantity', 'Batteries', 'ssd', 'Batteries Included?', 'Batteries Required?', 'Wattage', 'camera_resolution', 'Packaging', 'California residents', 'Voltage', 'Specification Met', 'Average Battery Life (in hours)', 'Audio-out Ports (#)', 'Power Source', 'Material', 'Usage', 'Certification', 'hdd', 'Special Features', 'Finish', 'Included Components', 'Style', 'W

In [26]:
for i in range(len(df)):
    keys = []
    for ind, k in enumerate(df.loc[i, 'specifications']):
        keys.append(k)
        if (('Size' in keys) and ('n_size' in keys)):
            del df.loc[i, 'specifications']['n_size']
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        keys.append(k)
        if (('hdd' in keys) and ('Hard Drive' in keys)):
            del df.loc[i, 'specifications']['hdd']
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        keys.append(k)
        if (('ram' in keys) and ('RAM' in keys)):
            del df.loc[i, 'specifications']['ram']
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        keys.append(k)
        if (('camera' in keys) and ('camera_resolution' in keys)):
            del df.loc[i, 'specifications']['camera']
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        keys.append(k)
        if (('power' in keys) and ('Wattage' in keys)):
            del df.loc[i, 'specifications']['Wattage']
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        keys.append(k)
        if (('power' in keys) and ('Output Wattage' in keys)):
            del df.loc[i, 'specifications']['Output Wattage']
            break

In [29]:
for i in range(len(df)):
    for ind, k in enumerate(df.loc[i, 'specifications']):
        if (k == 'n_size'):
            df.loc[i, 'specifications']['Size'] = df.loc[i, 'specifications'][k]
            del df.loc[i, 'specifications'][k]
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        if (k == 'camera'):
            df.loc[i, 'specifications']['camera_resolution'] = df.loc[i, 'specifications'][k]
            del df.loc[i, 'specifications'][k]
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        if (k == 'ram'):
            df.loc[i, 'specifications']['RAM'] = df.loc[i, 'specifications'][k]
            del df.loc[i, 'specifications'][k]
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        if (k == 'power'):
            df.loc[i, 'specifications']['Wattage'] = df.loc[i, 'specifications'][k]
            del df.loc[i, 'specifications'][k]
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        if (k == 'Output Wattage'):
            df.loc[i, 'specifications']['Wattage'] = df.loc[i, 'specifications'][k]
            del df.loc[i, 'specifications'][k]
            break
    for ind, k in enumerate(df.loc[i, 'specifications']):
        if (k == 'Screen Size'):
            df.loc[i, 'specifications']['Display Size'] = df.loc[i, 'specifications'][k]
            del df.loc[i, 'specifications'][k]
            break

In [73]:
def extract_unit(query, sen):
    start = re.search(query.lower(), sen.lower()).start()
    value = sen[0:start]
    unit = re.search(query.lower(), sen.lower()).group(0)
    return value, unit

lst = []
for i in range(len(df)):
    lst.append({})
    
df['spec_tup'] = lst

match = [r"ghz", r"mhz", r"gb", r"kb", r"tb", r"MP", r"db", r"lbs", r"pounds", r"ounces"]
unit_list = ["camera_resolution", "hdd", "Hard Drive", "RAM", "ram", "memory", "Memory Speed"]

for j in match:
    for i in range(len(df)):
            for ind, k in enumerate(df.loc[i, 'specifications']):
                val = df.loc[i, 'specifications'][k]
                if ((k in unit_list) and (j in val.lower())):
                    value, unit = extract_unit(j, val)
                    df.loc[i, 'spec_tup'][k] = (value, unit)

11315

In [79]:
df = df.drop(columns='spec_tup')

In [32]:
df.to_json('metadata/Meta_v4.json')